# Web scraping

In [50]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [123]:
#==================================================================================================================
def city_coordinates(CityName):
    url = f'https://en.wikipedia.org/wiki/{CityName}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    wiki_info = soup.find_all('table', class_  = 'infobox ib-settlement vcard')[0]
    
    latitude = wiki_info.find_all('span', class_ = 'latitude')[0].get_text()
    longitude = wiki_info.find_all('span', class_ = 'longitude')[0].get_text() 

    label = wiki_info.find_all('th', scope = 'row', class_ = 'infobox-label')
    data = wiki_info.find_all('td', class_ = 'infobox-data')
    box = wiki_info.find_all('tr', class_ = 'mergedtoprow')

    for i in range(len(box)):
        box_header_title = box[i].get_text()
        if re.search('Population', box_header_title): 

            date = box_header_title
            
            pop = box[i].find_next('td').get_text()
            # pop = re.split(r' |\[', pop)[0]
            break
    
    for i in range(len(data)):
        the_label = label[i].get_text().lower()
        if re.search('country', the_label):
            country = data[i].get_text().strip()
            break
    
    return [CityName, latitude, longitude, country, date, pop]

#==================================================================================================================
def to_decimal_coordinate(coordinates):
    latitude, longitude = coordinates['Latitude'], coordinates['Longitude']
    
    latitude, longitude = re.split('°|′|″', latitude), re.split('°|′|″', longitude)
    
    latitude = (float(latitude[0]) + float(latitude[1])/60)*((latitude[-1] == 'N') - (latitude[-1] == 'S'))
    longitude = (float(longitude[0]) + float(longitude[1])/60)*((longitude[-1] == 'E') - (longitude[-1] == 'W'))

    return [latitude, longitude]

#==================================================================================================================
def extract_the_year(info):
    date = info['YearDataRetrieved']
    # Find the characters between the parenthesis
    pattern = r'\(([^)]+)\)'
    match = re.search(pattern, date)
    # Extract the matching value
    date = match.group(1)

    # Find only integers with 4 digits
    pattern = r'\b(\d{4})\b'
    match = re.search(pattern, date)
    year = match.group(1)

    return year

#==================================================================================================================
def extract_the_population(info):
    # pop = info['Population']
    # Replace the character ',' by ''
    info = info.replace(',', '')
    
    # Find the firt characters that is an integer
    pattern = r'^\d+'
    match = re.search(pattern, info)
    
    return match.group()

In [165]:
# city = ['Berlin', 'Hamburg', 'Munich']
new_cities = ['Berlin', 'Hamburg', 'Munich', 'Ottawa', 'Lyon', 'Vienna', 'Brazilia', 'Dublin', 'Barcelona', 'Mexico City', 'Pretoria', 'Seoul', 'Tokyo', 'Paris', 'New_York_City']

scraping_city_df = pd.DataFrame()
for i in range(len(new_cities)):
    scraping_city_df.loc[i,['CityName', 'Latitude', 'Longitude', 'CountryName', 'YearDataRetrieved', 'Population']]  = city_coordinates(new_cities[i])
scraping_city_df

,CityName,Latitude,Longitude,CountryName,YearDataRetrieved,Population
0,Berlin,52°31′12″N,13°24′18″E,Germany,Population (2022 census)[4],"3,596,999"
1,Hamburg,53°33′N,10°00′E,Germany,Population (2023-12-31)[2],"1,964,021"
2,Munich,48°08′15″N,11°34′30″E,Germany,Population (2023-12-31)[2],"1,510,378"
3,Ottawa,45°25′29″N,75°41′42″W,Canada,Population (2021),"1,017,449 (4th)"
4,Lyon,45°46′N,4°50′E,France,"Population (2022)[5]520,774","520,774"
5,Vienna,48°12′30″N,16°22′21″E,Austria,"Population (2024)[1]2,014,614","2,014,614"
6,Brazilia,15°47′38″S,47°52′58″W,Brazil,Population (2022),"2,817,381[1][2] (3rd) [note 1]"
7,Dublin,53°21′00″N,06°15′37″W,Ireland,Population (2022),"592,713[3]"
8,Barcelona,41°23′N,2°11′E,Spain,Population (2018)[5],"1,620,343"
9,Mexico City,19°26′N,99°8′W,Mexico,Population (2020)[8],"9,209,944"


## Data Cleaning

### We convert the coordinates to decimal coordinates

In [166]:
scraping_city_df[['Latitude', 'Longitude']] = list(
    scraping_city_df[['Latitude', 'Longitude']]
    .apply(to_decimal_coordinate, axis = 1)
)
scraping_city_df

,CityName,Latitude,Longitude,CountryName,YearDataRetrieved,Population
0,Berlin,52.516667,13.400000,Germany,Population (2022 census)[4],"3,596,999"
1,Hamburg,53.550000,10.000000,Germany,Population (2023-12-31)[2],"1,964,021"
2,Munich,48.133333,11.566667,Germany,Population (2023-12-31)[2],"1,510,378"
3,Ottawa,45.416667,-75.683333,Canada,Population (2021),"1,017,449 (4th)"
4,Lyon,45.766667,4.833333,France,"Population (2022)[5]520,774","520,774"
5,Vienna,48.200000,16.366667,Austria,"Population (2024)[1]2,014,614","2,014,614"
6,Brazilia,-15.783333,-47.866667,Brazil,Population (2022),"2,817,381[1][2] (3rd) [note 1]"
7,Dublin,53.350000,-6.250000,Ireland,Population (2022),"592,713[3]"
8,Barcelona,41.383333,2.183333,Spain,Population (2018)[5],"1,620,343"
9,Mexico City,19.433333,-99.133333,Mexico,Population (2020)[8],"9,209,944"


### We extract the Year

In [167]:
scraping_city_df['YearDataRetrieved']  = scraping_city_df[['YearDataRetrieved']].apply(extract_the_year, axis = 1)
scraping_city_df

,CityName,Latitude,Longitude,CountryName,YearDataRetrieved,Population
0,Berlin,52.516667,13.400000,Germany,2022,"3,596,999"
1,Hamburg,53.550000,10.000000,Germany,2023,"1,964,021"
2,Munich,48.133333,11.566667,Germany,2023,"1,510,378"
3,Ottawa,45.416667,-75.683333,Canada,2021,"1,017,449 (4th)"
4,Lyon,45.766667,4.833333,France,2022,"520,774"
5,Vienna,48.200000,16.366667,Austria,2024,"2,014,614"
6,Brazilia,-15.783333,-47.866667,Brazil,2022,"2,817,381[1][2] (3rd) [note 1]"
7,Dublin,53.350000,-6.250000,Ireland,2022,"592,713[3]"
8,Barcelona,41.383333,2.183333,Spain,2018,"1,620,343"
9,Mexico City,19.433333,-99.133333,Mexico,2020,"9,209,944"


### We extract the number of populations

In [168]:
scraping_city_df['Population'] = scraping_city_df['Population'].apply(extract_the_population)

### Set the appropriate Data Types

In [172]:
# Set the appropriate Data types
scraping_city_df['Population'] = pd.to_numeric(scraping_city_df['Population'])
scraping_city_df['YearDataRetrieved'] = pd.to_numeric(scraping_city_df['YearDataRetrieved'])
scraping_city_df.info()

,CityName,Latitude,Longitude,CountryName,YearDataRetrieved,Population
0,Berlin,52.516667,13.400000,Germany,2022,3596999
1,Hamburg,53.550000,10.000000,Germany,2023,1964021
2,Munich,48.133333,11.566667,Germany,2023,1510378
3,Ottawa,45.416667,-75.683333,Canada,2021,1017449
4,Lyon,45.766667,4.833333,France,2022,520774
5,Vienna,48.200000,16.366667,Austria,2024,2014614
6,Brazilia,-15.783333,-47.866667,Brazil,2022,2817381
7,Dublin,53.350000,-6.250000,Ireland,2022,592713
8,Barcelona,41.383333,2.183333,Spain,2018,1620343
9,Mexico City,19.433333,-99.133333,Mexico,2020,9209944


# Connecting to MySQL

In [76]:
schema = "metropolis"
host = "127.0.0.1"
user = "root"
password = "mysql_password"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## Creating the tables with Python

### Table `city`

In [134]:
city_df = pd.DataFrame(data = scraping_city_df['CityName'].unique(),
                       columns = ['CityName'])
city_df

,CityName
0,Berlin
1,Hamburg
2,Munich
3,Ottawa
4,Lyon
5,Vienna
6,Brazilia
7,Dublin
8,Barcelona
9,Mexico City


### Table `geo_loc`

In [69]:
geo_loc_df = pd.DataFrame(data = scraping_city_df[['Latitude', 'Longitude']],
                         columns = ['Latitude', 'Longitude'])
geo_loc_df

,Latitude,Longitude
0,52.52,13.405
1,53.55,10.0
2,48.1375,11.575
3,45.424722,-75.695
4,45.766667,4.833333
5,48.208333,16.3725
6,-15.793889,-47.882778
7,53.35,-6.260278
8,41.383333,2.183333
9,19.433333,-99.133333


## Creating the matching tables on MySQL

## Sending it to MySQL

In [135]:
city_df.to_sql('city',
                  if_exists='append',
                  con=connection_string,
                  index=False)

15

## Receiving from MySQL

In [136]:
query = '''
SELECT *
FROM city
'''
city_from_sql = pd.read_sql(sql = query, con=connection_string)
city_from_sql

,CityId,CityName
0,1,Berlin
1,2,Hamburg
2,3,Munich
3,4,Ottawa
4,5,Lyon
5,6,Vienna
6,7,Brazilia
7,8,Dublin
8,9,Barcelona
9,10,Mexico City


## Sending other tables to MySQL

### Country

In [140]:
country_df = scraping_city_df.merge(city_from_sql, on = 'CityName', how = 'left')
country_df.drop(columns = ['CityName', 'Latitude', 'Longitude', 'YearDataRetrieved', 'Population'],
               inplace = True)
# country_df

In [139]:
country_df.to_sql('country',
                  if_exists='append',
                  con=connection_string,
                  index=False)

15

### Geo Localisation

In [144]:
geo_loc_df = scraping_city_df.merge(city_from_sql, on = 'CityName', how = 'left')
geo_loc_df.drop(columns = ['CountryName', 'YearDataRetrieved', 'Population', 'CityName'],
               inplace = True)
geo_loc_df

,Latitude,Longitude,CityId
0,52.52,13.405,1
1,53.55,10.0,2
2,48.1375,11.575,3
3,45.424722,-75.695,4
4,45.766667,4.833333,5
5,48.208333,16.3725,6
6,-15.793889,-47.882778,7
7,53.35,-6.260278,8
8,41.383333,2.183333,9
9,19.433333,-99.133333,10


In [145]:
geo_loc_df.to_sql('geolocation',
                  if_exists='append',
                  con=connection_string,
                  index=False)

15

### Population

In [150]:
population_df = scraping_city_df.merge(city_from_sql, on = 'CityName', how = 'left')
population_df.drop(columns = ['CountryName', 'Latitude', 'Longitude', 'CityName'],
               inplace = True)
population_df

,YearDataRetrieved,Population,CityId
0,2022,3596999,1
1,2023,1964021,2
2,2023,1510378,3
3,2021,1017449,4
4,2022,520774,5
5,2024,2014614,6
6,2022,2817381,7
7,2022,592713,8
8,2018,1620343,9
9,2020,9209944,10


In [162]:
population_df.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

15